## Project 6: Working with Geocoded Data

### Building Maps in geopandas
In this lesson we will download COVID-19 data from data.world. We will normalize the data to compare spread between counties. Were we to simply plot the total number of cases or deaths by county, the results would be biased as counties with larger populations would likely have more cases and more deaths. We will observe how the spread developed across the country, starting in the northeast, eventually making its way to other regions.

**Installing geopandas**

Although there is a geopandas installation available using the conda install command in you command line shell, that package is incomplete for our purposes. We will need to install dependencies - in this order: GDAL,Fiona, and Shapely - for geopandas before installing geopandas. I have included the .whl files for each of these packages in the same folder is this notebook. Download the files and save them to your local folder. To install, use the command:

> pip install filename

If you are using a machine for which you are not the administrator, use the command:

> pip install filename --user

Install the libraries using these commands from an administrator shell or using --user at the end of the statement:

> pip install GDAL-3.1.4-cp37-cp37m-win_amd64.whl

> pip install Fiona-1.8.17-cp37-cp37m-win_amd64.whl

> pip install Shapely-1.7.1-cp37-cp37m-win_amd64.whl

If you are using Python 3.8, use the following wheels:

> pip install GDAL-3.1.4-cp38-cp38-win_amd64.whl

> pip install Fiona-1.8.17-cp38-cp38-win_amd64.whl

> pip install Shapely-1.7.1-cp38-cp38-win_amd64.whl

If you are using a newer version of python and it's giving you issues installing GDAL, Fiona and Shapely, use the command line or powershell to downgrade/upgrade the version to 3.7:
> conda install python =3.7 anaconda=custom

Once you've successfully changed the version to 3.7, you can then run the pip commands above to install GDAL, Fiona and Shapely. 

If you are using a mac, you may install the appropriate module by selecting the version.

> pip install -v GDAL==3.1.4

> pip install -v Fiona==1.8.17

> pip install -v Shapely==1.7.1

Finally, install geopandas:

> pip install geopandas

### Downloading the COVID-19 data

We will use two datasets. First, we will import a shapefile to use with geopandas, which we will later use to generate a county level map that tracks COVID-19. The shapefile is provide for you in the Github folder housing this post. You can also download shapefiles from the [U.S. Census website](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html). We will download Johns Hopkins's COVID-19 data from the Associated Press's [account](https://data.world/associatedpress/johns-hopkins-coronavirus-case-tracker) at data.world using their [Python module](https://data.world/integrations/python). Follow [these instructions](https://github.com/datadotworld/data.world-py/) to install the datadotworld module and access their API.

> Datadotworld may be useful efficiently collecting data for class projects, so keep this libary in mind as you make plans for your project.

**Note:**
> Checkout this **[article](https://www.ndsu.edu/centers/pcpe/news/detail/58432/)** from NDSU PCPE website for some additional information on COVID-19 data analysis and visualization

In [ ]:
#COVID19Map.py
### import all modules that we will use in this lesson
import numpy as np
import pandas as pd
import geopandas

# datadotworld will connect to the John's hopkin's python API
import datadotworld as dw

# We won't actually use datetime directly. Since the dataframe index will use 
# data formatted as datetime64, I import it in case I need to use the datetime module to troubleshoot later 
import datetime

# you could technically call many of the submodules from matplotlib using mpl., 
#but for convenience we explicitly import submodules. These will be used for constructing visualizations
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# If you choose to make a dynamic visualization for the homework
from matplotlib.animation import FuncAnimation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.ticker as mtick


In [ ]:
# creating a function to download our covid data from John Hopkins website
def import_covid_data(filename, FIPS_name):
    # Load COVID19 county data using datadotworld API
    # Data provided by Johns Hopkins, file provided by Associated Press
    dataset = dw.load_dataset(
        "associatedpress/johns-hopkins-coronavirus-case-tracker",
        auto_update = True)
    
    # the dataset includes multiple dataframes. We will only use #2
    covid_data = dataset.dataframes["2_cases_and_deaths_by_county_timeseries"]
    
    # Include only oberservation for political entities within states
    # i.e., not territories, etc... drop any nan fip values with covid_data[FIPS_name] > 0
    # so, look at covid data df named covid_data, 
    # filter observations by the FIPS_name column where it's less than 57000 or greater than 0
    # the first 2 digits of the fip_code identify the state (google county fips codes)
    # the following 3 digits identify the county
    # Note that fips_code > 57000 gives you territories and not actual states (google fips code)
    covid_data = covid_data[covid_data[FIPS_name] < 57000]
    covid_data = covid_data[covid_data[FIPS_name] > 0]

    # Transform FIPS codes into integers (not floats)
    covid_data[FIPS_name] = covid_data[FIPS_name].astype(int)
    
    # set the index as fips and date. The index will make it easy pandas to access/inteprete the data and df
    # inplace = true says make the change in the same df. No need to create a new df with the change
    covid_data.set_index([FIPS_name, "date"], inplace = True)
    
    # Prepare a column for state abbreviations. We will draw these from a
    # state_dict dictionary created in the next step.
    
    # first, let's initialize the column by creating a blank string ("") since the column will only have str. values
    covid_data["state_abr"] = ""
    
    # for each entry, we'll use the abbreviations in the state_dict. 
    # where the state name = state_x, use the abbreviation from State_dict that corresponds to that state.
    for state, abr in state_dict.items():
        
        # .loc[row(s), col] says call covid data, for all rows where the state == whatever state we are on, 
        # enter a value under state abbr. for all those rows
        covid_data.loc[covid_data["state"] == state, "state_abr"] = abr
        
    # Create "Location" which concatenates county name and state abbreviation 
    # save location name as County, Abr (e.g. Cass, ND)
    covid_data["Location"] = covid_data["location_name"] + ", " + covid_data["state_abr"]

    return covid_data


# create Geo dataframe
def import_geo_data(filename, index_col = "Date", FIPS_name = "FIPS"):
    
    # use geopandas to import county level shapefile
    map_data = geopandas.read_file(filename = filename, index_col = index_col)
    
    # rename fips code to match variable name in COVID-19 data
    map_data.rename(columns={"State":"state"}, inplace = True)
    
    # Combine statefips and county fips to create a single fips value
    # that identifies each particular county without referencing the state separately
    map_data[FIPS_name] = map_data["STATEFP"].astype(str) + map_data["COUNTYFP"].astype(str)
    map_data[FIPS_name] = map_data[FIPS_name].astype(np.int64)
    
    # set FIPS as index
    map_data.set_index(FIPS_name, inplace=True)
    
    return map_data


# I include this dictionary to convenienlty cross reference state names and state abbreviations.
state_dict = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 
              'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT','Delaware': 'DE', 
              'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 
              'Idaho': 'ID', 'Illinois': 'IL','Indiana': 'IN', 'Iowa': 'IA','Kansas': 'KS', 
              'Kentucky': 'KY','Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
              'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 
              'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 
              'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
              'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 
              'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 
              'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


In [ ]:
fips_name = "fips_code"
covid_filename = "COVID19DataAP.csv"

# rename_FIPS matches map_data FIPS with COVID19 FIPS name
covid_data = import_covid_data(filename = covid_filename, FIPS_name = fips_name)
covid_data

We can see that counties are identified by a state, with reference to the states fip code, and by a county level fip code. Together, these make the total fip code. We will use the fip code to align the data frames.

Notice also that each observation is associated with an entry under **geometry.** Each entry has identified with it the shape and location of the county identified by a series of coordinates representing the perimeter of the county.

Next, let's call the COVID-19 data.

In [ ]:
map_data = import_geo_data(filename = "countiesWithStatesAndPopulation.shp",
                           index_col = "Date", FIPS_name= fips_name)

#covid_data = import_covid_data(filename = covid_filename, FIPS_name = fips_name)
map_data

In [ ]:
# we can print a particular record in the df. 38017 is the fips code index and we want to return 
# the record in the geometry column that also corresponds to fips_code 21007. Since the geometry column
# is made of coordinates, it plots a polygon using the coordinates which is actually the shape of the county

map_data.loc[38017, "geometry"]

In [ ]:
# if you prefer seeing the text value rather than the shape of the coordinate, you can use the "print" command
print(map_data.loc[38017, "geometry"])

In [ ]:
# We can also plot all the data at once using df.plot(). However, we'll want to specify map parameters
# Note: you might need to "pip install descartes" to plot the entire map (or all the polygons what makes up the map)
map_data.plot()

The COVID-19 data also includes a fip_code column in the index. Notice that the index also includes the date. To transform the covid_data into a geodataframe, we will need to duplicate the geodataframe for every date in the multi-index of covid_data. Once the geodataframe has been developed with an index that matches the covid_data, we can add every column from covid_data to the geodataframe by using a for loop to cycle three each key and column of data (key, val) in covid_data.

In [ ]:
#COVID19Map.py
# . . .
# create pandas geo df
def create_covid_geo_dataframe(covid_data, map_data, dates):
    # create geopandas dataframe with multiindex for date
    # original geopandas dataframe had no dates, so copies of the df are 
    # stacked vertically, with a new copy for each date in the covid_data index
    #(dates is a global)
    i = 0
    for date in dates:
        
        # select county observations from each date in dates
        df = covid_data[covid_data.index.get_level_values("date")==date]
        
        # use the fips_codes from the slice of covid_data to select counties
        # from the map_data index,making sure that the map_data index matches
        # the covid_data index
        counties = df.index.get_level_values("fips_code")
        
        # call a slice of the geodataframe that includes the counties that are in covid_data
        agg_df = map_data.loc[counties]
       
        # each row should reflect the date so that it is aligned with covid_data
        agg_df["date"] = date
        if i == 0:
            
            # create the geodataframe, select coordinate system (.crs) to match map_data.crs
            matching_gpd = geopandas.GeoDataFrame(agg_df, crs = map_data.crs)
            i += 1
        else:
            
            # after initial geodataframe is created, stack a dataframe for each date in dates. 
            # Once completed, index of matching_gpd will match index of covid_data
            matching_gpd = matching_gpd.append(agg_df, ignore_index = False) 
            
    # Set mathcing_gpd index as["fips_code", "date"], liked covid_data index
    matching_gpd.reset_index(inplace=True)
    matching_gpd.set_index(["fips_code","date"], inplace = True)
    
    # add each column from covid_data to mathcing_gpd
    for key, val in covid_data.items():
        matching_gpd[key] = val
    return matching_gpd       

# . . . to end of script . . .

In [ ]:
# create a list of dates
# dates will be used to create a geopandas DataFrame with multiindex 
# set(lst) removes duplicate entries
dates = sorted(list(set(covid_data.index.get_level_values("date"))))
covid_data = create_covid_geo_dataframe(covid_data, map_data, dates)

covid_data

Next, we will normalize the data by population using the create_new_vars() function. We will use the population data that I have included with the .shp file under the key "Population". If you have downloaded the shapefile directly from the census website, than you might prefer to use "total_population", which is included in the Associated Press's COVID-19 data.

Calculate deaths as a percent of population, then multiply that percentage by $10^6$ (one million) to calculate deaths per million. In addition to normalizing the data by population, we will also calculate the 7 day running average of this variable using the .rolling() method followed by .mean().

In [ ]:
#COVID19Map.py
# . . .
def create_new_vars(covid_data, moving_average_days):
    
    # use a for loop that performs the same operations on data for cases and for deaths
    for key in ["cases", "deaths"]:
        
        # create a version of the key with the first letter of each word capitalized
        cap_key = key.title()
        covid_data[cap_key + " per Million"] = covid_data["cumulative_" + key]\
            .div(covid_data["total_population"]).mul(10 ** 6)
        
        # generate daily data normalized per million population by taking the daily difference within each
        # entity (covid_data.index.names[0]), dividing this value by population and multiplying that value
        # by 1 million 10 ** 6
        # can also use .groupby("figs_code")
        covid_data["Daily " + cap_key + " per Million"] =\
            covid_data["cumulative_" + key].groupby(covid_data.index.names[0]).diff(1)\
            .div(covid_data["total_population"]).mul(10 ** 6)
        
        # taking the rolling average; choice of number of days is passed as moving_average_days
        # .rolling lets python hold the variables but not record it then take the mean of the values
        covid_data["Daily " + cap_key + " per Million MA"] = covid_data["Daily " + cap_key + " per Million"]\
                   .rolling(moving_average_days).mean()
# . . .
moving_average_days = 7
create_new_vars(covid_data, moving_average_days)

In [ ]:
covid_data

We are almost ready to create our first map! Much of the same structure for plotting other visualizations in matplotlib using pandas also applies to using geopandas to create maps. We will initiate a figure using plt.subplots(). Since we are plotting time series data, we will need to plot only one date at a time. Let's start with the most recent data available.

To start, we need to select data from counties within the 48 states, otherwise Alaska and Hawaii will be include. This allows the area represented to be much smaller. The measures that are slected are rough estimates, but all that matters is that the 48 states reside within the boundaries chosen. Similar to the data_processed variable, I include a map_bounded variable so that this computation will not be repeated when the script is run multiple times.

In [ ]:
#COVID19Map.py
#. . . 

# only include observations within these boundaries (minx, maxx, miny, maxy)
# this will shrink the size of the map
# maxx and maxy refer to max x and y coordinates respectively
# minx and miny refer to min x and y coordinates
def select_data_within_bounds(data, minx, miny, maxx, maxy):
    data = data[data.bounds["maxx"] <= maxx]
    data = data[data.bounds["maxy"] <= maxy]
    data = data[data.bounds["minx"] >= minx]
    data = data[data.bounds["miny"] >= miny]
    
    return data
# . . . to end of script

In [ ]:
# choose most recent date in data
date = dates[-1]

# choose map bounds
if "map_bounded" not in locals():
    # define boundary coordinates
    minx = -127
    miny = 23
    maxx = -66
    maxy = 48
    
    # call function and redefine data
    covid_data = select_data_within_bounds(covid_data, minx, miny, maxx, maxy)
    map_bounded = True

In [ ]:
# plot map
fig, ax = plt.subplots(figsize=(18,8),
                       subplot_kw = {'aspect': 'equal'})   

plt.rcParams.update({"font.size": 30})
plt.xticks(fontsize = 25)
plt.yticks(fontsize = 25)
key = "Deaths per Million"

# define df for subset of data to only include datas we're concerned about
df = covid_data[covid_data.index.get_level_values("date") == date]
df.plot(ax=ax, cax = ax, column=key, linewidth = 0.5, edgecolor='lightgrey')

ax.set_title(str(date) + "\n" + "COVID-19 in the U.S.", fontsize = 30)

The result looks okay. A few things can be improved. First, it's usually a good idea to select a colormap that spans a narrow range of colors. For this purpose, we will choose to use the colorbar defined by "Reds" in matplotlib. Color choice also conveys affect to the reader. A choice of color, say "Blues", might strike the viewer as conveying neutral content. Our choice of red conveys that areas that are darker shades of red tend to have been more greatly impacted by the COVID-19. The variable "Deaths per Million" normalizes death by population, allowing for a fair comparison between counties.

We will also want to limit the variety of shades used for representation. It is easier to interpret data that has been divided into discrete categories. We will choose to use 4 categories representing ranges of increasing size and breadth.

Since we are plotting levels of values, it will be useful to compare logged values. This will give us a sense that some areas are doing better or worse by orders of magnitude, depending on the plot, about 10 to 20 times better or worse given each change in color. To have colors represent a range of logged values, we use the cm.colors.LogNorm() and apply this with plt.cm.ScalarMappable(). We explicitly identify min and max values as this will be useful to us when we plot animations over time later in this lesson.

Finally, in the title, we identify the date, region, and the category of data represented.

In [ ]:

#COVID19Map.py

fig, ax = plt.subplots(figsize=(18,8),
        subplot_kw = {'aspect': 'equal'}) 

plt.rcParams.update({"font.size": 30})
plt.xticks(fontsize = 25)
plt.yticks(fontsize = 25)
key = "Deaths per Million"

# change colors, divide into 4 distinct colors
# if you google matplotlib color maps, you'll have names of color themes to use. 
# That's how we got "Reds" color theme for the map. 
# We could replace "Reds" with Purples, Greys, OrRd or any other color theme found here 
# https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
cmap = cm.get_cmap('Reds', 4)

# let's set our vmin log value to 1 and vmax value to the highest value in our df.

vmin = 1 
vmax = df[key].max()

# we'll use the vmin and vmax to create the color bar legend as seen two maps below
# norm defines the range of the logged values
norm = cm.colors.LogNorm(vmin=vmin, vmax =vmax)

# scalarMappable uses that range in the colorbar
plt.cm.ScalarMappable(cmap=cmap, norm=norm)
df = covid_data[covid_data.index.get_level_values("date") == date]
df.plot(ax=ax, cax = ax, column=key, vmin=vmin ,vmax = vmax, 
        cmap = cmap, legend=False, linewidth=.5, edgecolor='lightgrey', 
        norm = norm)

ax.set_title(str(date) + "\n" + "COVID-19 in the U.S.", fontsize = 30)

We also want to include a colorbar so that the viewer knows what values are conveyed by map colors. This requires the addition of several parameters that allow for inclusion of a colorbar legend. To construct the colorbar, we need to call make_axes_locatable(ax), then use this to create a colorbar axis, cax. Using cax and cmap, created in the block of code executed in the previous section, we are able to create a colorbar. To control the value format, we create a list of the axis values, making sure that they are integers (not floats). Then, identify the newly created matplotlib objects in the .plot() method.

Since the color axis is logged, technically it doesn't include a representation for 0. When passing the dataframe, replace all 0 values with 1 in order to transform the white counties into beige. If you were to use this map, be sure to note that you have made this adjustment.

In [ ]:
fig, ax = plt.subplots(figsize=(18,8),
        subplot_kw = {'aspect': 'equal'})   
plt.rcParams.update({"font.size": 30})
plt.xticks(fontsize = 25)
plt.yticks(fontsize = 25)
key = "Deaths per Million"
df = covid_data[covid_data.index.get_level_values("date")==date]
cmap = cm.get_cmap('Reds', 4)
vmin = 1 
vmax = df[key].max()
norm = cm.colors.LogNorm(vmin=vmin, vmax =vmax)

############## Add Colorbar ################
sm = cm.ScalarMappable(cmap=cmap, norm=norm)

# empty array for the data range
sm._A = []

# prepare space for colorbar
divider = make_axes_locatable(ax)
size = "5%" 
cax = divider.append_axes("right", size = size, pad = 0.1)

# add colorbar to figure
cbar = fig.colorbar(sm, cax=cax, cmap = cmap)
cbar.ax.tick_params(labelsize=18)
vals = list(cbar.ax.get_yticks())
vals.append(vmax)

# format colorbar values as int
cbar.ax.set_yticklabels([int(x) for x in vals])
cbar.ax.set_ylabel(key, fontsize = 20)

df.plot(ax=ax, cax = ax, column=key, vmin=vmin ,vmax = vmax, 
             cmap = cmap, legend=False, linewidth=.5, edgecolor='lightgrey', 
             norm = norm)
ax.set_title(str(date)[:10] + "\n" + "COVID-19 in the U.S.", fontsize = 30)

We've successfully made a quality map. Next, let's use a for loop to create maps for all for categories that we plotted in the previous post. This is simple. First we define a list of keys, and iterate over each key.

In [ ]:

keys = ["Cases per Million", "Deaths per Million", 
        "Daily Cases per Million MA", "Daily Deaths per Million MA"]
for key in keys:
    # identify whether or not to log values for color axis
    # if daily rates, do not log. Only log totals.
    log = False if "Daily" in key else True
    fig, ax = plt.subplots(figsize=(18,8),
        subplot_kw = {'aspect': 'equal'})   
    plt.rcParams.update({"font.size": 30})
    plt.xticks(fontsize = 25)
    plt.yticks(fontsize = 25)
    
    # this time we replace 0 values with 1
    # so that these values show up as beige instead of as white
    # when color axis is logged
    df = covid_data[covid_data.index.get_level_values("date")==date].replace(0,1)
    
    # set range of colorbar
    vmin = 1 if log else 0 
    vmax = df[key].max()
    
    # choose colormap
    cmap = cm.get_cmap('Reds', 4)
    
    # format colormap
    if log:
        norm = cm.colors.LogNorm(vmin=vmin, vmax =vmax)
    else:
        norm = cm.colors.Normalize(vmin = vmin, vmax = vmax)
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    
    # empty array for the data range
    sm._A = []
    
    # prepare space for colorbar
    divider = make_axes_locatable(ax)
    size = "5%" 
    cax = divider.append_axes("right", size = size, pad = 0.1)
    
    # add colorbar to figure
    cbar = fig.colorbar(sm, cax=cax, cmap = cmap)
    cbar.ax.tick_params(labelsize=18)
    vals = list(cbar.ax.get_yticks())
    vals.append(vmax)

    # format colorbar values as int
    cbar.ax.set_yticklabels([int(x) for x in vals])
    cbar.ax.set_ylabel(key, fontsize = 20)


    df.plot(ax=ax, cax = cax, column=key, vmin=vmin ,vmax = vmax, 
                 cmap = cmap, legend=False, linewidth=.5, edgecolor='lightgrey', 
                 norm = norm)
    ax.set_title(str(date)[:10] + "\n" + "COVID-19 in the U.S.", fontsize = 30)

    plt.show()
    plt.close()

In [ ]:

# initiate/create empty pdf file
pp = PdfPages("COVID-19Maps.pdf")

keys = ["Cases per Million", "Deaths per Million", 
        "Daily Cases per Million MA", "Daily Deaths per Million MA"]

# take dates as a set, then make a list out of it, then sort it. 
dates = sorted(
    list(
        set(covid_data.index.get_level_values("date"))))

# only include every 7th day. It's obviosly not required but
# it should speed up how fast the program runs the code  
dates = dates[::7]
for key in keys:
    # set range of colorbar
    vmin = 1   
    vmax = df[key].max()
        
    for date in dates:
        # identify whether or not to log values for color axis
        # if daily rates, do not log. Only log totals.
        log = False if "Daily" in key else True
        fig, ax = plt.subplots(figsize=(18,8), 
                               subplot_kw = {'aspect': 'equal'})  
        
        plt.rcParams.update({"font.size": 30})
        plt.xticks(fontsize = 25)
        plt.yticks(fontsize = 25)

        # this time we replace 0 values with 1
        # so that these values show up as beige instead of as white
        # when color axis is logged
        df = covid_data[covid_data.index.get_level_values("date")==date]

        # choose colormap
        cmap = cm.get_cmap('Reds', 4)

        # format colormap
        if log:
            norm = cm.colors.LogNorm(vmin=vmin, vmax =vmax)
        else:
            norm = cm.colors.Normalize(vmin = vmin, vmax = vmax)
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)

        # empty array for the data range
        sm._A = []

        # prepare space for colorbar
        divider = make_axes_locatable(ax)
        size = "5%" 
        cax = divider.append_axes("right", size = size, pad = 0.1)

        # add colorbar to figure
        cbar = fig.colorbar(sm, cax=cax, cmap = cmap)
        cbar.ax.tick_params(labelsize=18)
        vals = list(cbar.ax.get_yticks())
        vals.append(vmax)

        # format colorbar values as int
        cbar.ax.set_yticklabels([int(x) for x in vals])
        cbar.ax.set_ylabel(key, fontsize = 20)


        df.plot(ax=ax, cax = cax, column=key, vmin=vmin ,vmax = vmax, 
                     cmap = cmap, legend=False, linewidth=.5, edgecolor='lightgrey', 
                     norm = norm)
        ax.set_title(str(date)[:10] + "\n" + "COVID-19 in the U.S.", fontsize = 30)

        plt.show()
        
        # save in pdf file
        pp.savefig(fig, bbox_inches="tight")
        plt.close()
        
pp.close()

In [ ]:
# only select data that falls within bounds of coordinates
def select_data_within_bounds(data, minx, miny, maxx, maxy):
    data = data[data.bounds["maxx"] <= maxx]
    data = data[data.bounds["maxy"] <= maxy]
    data = data[data.bounds["minx"] >= minx]
    data = data[data.bounds["miny"] >= miny]
    
    return data

def plot_map(*kwargs):
    plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    plot_df = df[df.index.get_level_values("date")==date]
    plot_df.plot(ax=ax, cax = ax, column=key, vmin=vmin ,vmax = vmax, 
                 cmap = cmap, legend=False, linewidth=.5, edgecolor='lightgrey', 
                 norm = norm)
    ax.set_title(str(date)[:10] + "\n" + "COVID-19 in the U.S.", fontsize = 30)
    
def init(*kwargs):
    size = "5%"     
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    # empty array for the data range
    sm._A = []
    # add the colorbar to the figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size = size, pad = 0.1)
    # add colorbar to figure
    cbar = fig.colorbar(sm, cax=cax, cmap = cmap)
    cbar.ax.tick_params(labelsize=18)
    vals = list(cbar.ax.get_yticks())
    vals.append(vmax)
    if "Daily" not in key: vals[0] = 0 
    # format colorbar with logged or observed values
    if log:
        cbar.ax.yaxis.set_major_formatter(mtick.LogFormatter())
    else:
        cbar.ax.yaxis.set_major_formatter(mtick.Formatter())
    # format colorbar values as int
    cbar.ax.set_yticklabels([int(x) for x in vals])
    cbar.ax.set_ylabel(key, fontsize = 20)
    
date = dates[-1]
keys = ["Cases per Million", "Deaths per Million", 
        "Daily Cases per Million MA", "Daily Deaths per Million MA"]

if "map_bounded" not in locals():
    minx = -127
    miny = 23
    maxx = -58
    maxy = 54
    covid_data = select_data_within_bounds(covid_data, minx, miny, maxx, maxy)
    map_bounded = True


for key in keys:
    log = False if "Daily" in key else True
    # this time we replace 0 values with 1
    # so that these values show up as beige  instead of as white
    # when color axis is logged
    vmin = 1 if log else 0 
    vmax = df[key][df.index.get_level_values("date") == date].max()
    # Create colorbar as a legend
    cmap = cm.get_cmap('Reds', 4)
    if log:
        norm = cm.colors.LogNorm(vmin=vmin, vmax =vmax)
    else:
        norm = cm.colors.Normalize(vmin = vmin, vmax = vmax)

    fig, ax = plt.subplots(figsize=(18,8),
        subplot_kw = {'aspect': 'equal'})   
    plt.rcParams.update({"font.size": 30})
    plt.xticks(fontsize = 25)
    plt.yticks(fontsize = 25)
    # the functions will unpack the tuple. The same names variable names
    # are used in the function
    kwargs = (df, key, log, date, fig, ax, cmap, norm, vmin, vmax)
    init(kwargs)
    plot_map(kwargs)
    
    plt.show()    

    plt.close()

Now, let's take a look at another set of data. This is unemployment data

In [ ]:
#import unemployment data from csv
# CSV file was downloaded from www.bls.gov/lau

# encoding is used since the text itself is formatted in a particular way
u_data = pd.read_csv(
    "countyUnemploymentData.csv", encoding = "latin1", parse_dates = True)

# drop observations with missing fips codes
# .index says show us the index
index = u_data["fips_code"].dropna(axis = 0).index

# .loc says access U_data for all index
u_data = u_data.loc[index]

u_data["fips_code"] = u_data["fips_code"].astype(int)
u_data.set_index(["fips_code", "date"], inplace = True)

In [ ]:
u_data

In [ ]:
#We can view all the geo-coordinates of county 1069
u_data.loc[1069, "geometry"]


#u_data.loc[1069, "geometry"].loc["Feb-20"]

In [ ]:
#COVID19Map.py
#. . . 
# this function is a simplified version of the Create_Covid_geo_dataframe function we created above
def create_merged_geo_dataframe(data, map_data, dates):
    data_frame_initialized = False
    # identify the counties before the for loop will make the code to run faster
    counties = sorted(
        list(
            set(data.index.get_level_values("fips_code"))))
    for date in dates:
       
        # select county observations from each date in dates
        # select only the subset of counties in the map that
        # are also present in the covid_data
        agg_df = map_data.loc[counties]
        agg_df["date"] = date
        if data_frame_initialized == False:
            
            #Create new dataframe
            matching_gpd = geopandas.GeoDataFrame(agg_df, crs = map_data.crs)
            data_frame_initialized = True        
        
        else:
            # or stack thenew  data frame and the dataframe that was initialized at
            # i = 0
            matching_gpd = matching_gpd.append(agg_df, ignore_index = False)
            
    # reset index, then set to [fips, date]
    matching_gpd.reset_index(inplace=True)
    matching_gpd.set_index(["fips_code", "date"], inplace = True)
    
    for key, val in data.items():
        matching_gpd[key] = val
    return matching_gpd

In [ ]:
# choose the dates
dates = ["Aug-19",
         "Sep-19",
         "Oct-19",
         "Nov-19",
         "Dec-19",
         "Jan-20",
         "Feb-20",
         "Mar-20",
         "Apr-20",
         "May-20",
         "Jun-20",
         "Jul-20",
         "Aug-20",
         "Sep-20"]

# another way of selecting the dates.
#dates = list(set(u_data.index.get_level_values("date")))

u_data = create_merged_geo_dataframe(u_data, map_data, dates)

In [ ]:
u_data

In [ ]:
# choose map bounds

#if "u_map_bounded" not in locals():
minx = -127
miny = 23
maxx = -66
maxy = 50
u_data = select_data_within_bounds(u_data, minx, miny, maxx, maxy)
u_map_bounded = True

In [ ]:
counties = sorted(list(set(u_data.index.get_level_values("fips_code"))))

In [ ]:
key = "Unemployment Rate"

# csv saved data as string, transform to float
u_data[key] = u_data[key].astype(float)

# create new pdf
pp = PdfPages("County Unemployment Rate.pdf")
for date in dates:
    fig, ax = plt.subplots(figsize=(19,9),
                          subplot_kw = {"aspect":"equal"})
    plt.rcParams.update({"font.size": 30})

    plt.xticks(fontsize = 25)
    plt.yticks(fontsize = 25)

    # set range of color bar. Min is 0, max is 20 and divide it to 4 so color bar will be 5,10,15,20
    vmin = 0
    #vmax = u_data[key].fillna(0).max()
    vmax = 20
    cmap = cm.get_cmap("Reds", 4)
    norm = cm.colors.Normalize(vmin=vmin, vmax=vmax)
    ### add colorbar
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    # empty array for the data range
    sm.A = []
    # prepare space for colorbar
    divider = make_axes_locatable(ax)
    size = "5%"
    # we'll place the colorbar on the right of the map
    cax = divider.append_axes("right", size = size, pad = .1)
    # add colorbar to figure
    cbar = fig.colorbar(sm, cax=cax, cmap= cmap)
    cbar.ax.tick_params(labelsize=18)
    vals = list(cbar.ax.get_yticks())
    vals.append(vmax)
    cbar.ax.set_yticklabels(vals)#[int(x) for x in vals])
    cbar.ax.set_ylabel(key, fontsize=20)

    # select data only from date
    df = u_data[u_data.index.get_level_values("date") == date]#.dropna(axis=0)
    df.plot(ax=ax, cax=ax, column = key,
            vmin=vmin, vmax=vmax,
            cmap=cmap, legend=False, 
            linewidth = .5, edgecolor="lightgrey",norm=norm)
    ax.set_title(date.replace("-", " 20"))
    plt.show()
    pp.savefig(fig, bbox_inches = "tight")
    plt.close()
    
# close your pdf
pp.close()

In [ ]:
# Normalize Unemployment Feb-20 == 1
key = "Unemployment Rate"

# df.copy() makes a copy of the dataframe
# n_u_data represents normalized data and we'll normalize the data to 1
n_u_data = u_data.copy()

# go through data for every county for the key and divide the value by the value of the key
# at the date that you would to normalize to 1
# what this does is, it makes Febrary the standard to which data for all other months will be compared to
# so, we are dividing all the employer data for each month by the unemployment data in february.
# this will help us see the changes that occured in each month relative to February. 
for county in counties:
    n_u_data[key][county] = n_u_data.loc[county, key].div(n_u_data.loc[county, "Feb-20"][key])
    
# alternatively, take the difference between the observed rate and the Feb rate:
# for county in countries:
#     n_u_data[key][county] = n_u_data.loc[county, key].sub(n_u_data.loc[county, "Feb-20"][key])

In [ ]:
pp = PdfPages("Normalized County Unemployment Rate.pdf")

for date in dates:
    # accomplishes same outcome as date.replace("-", " 20")
    title_date = date[:-3]+" 20" + date[-2:]
    fig, ax = plt.subplots(figsize=(19,9),
                          subplot_kw = {"aspect":"equal"})
    plt.rcParams.update({"font.size": 30})

    plt.xticks(fontsize = 25)
    plt.yticks(fontsize = 25)
    
    # set range color bar values
    vmin = 0 # n_u_data["Unemployment Rate"].min()
    vmax = 10
    cmap = cm.get_cmap("Reds", 10)
#    cmap = cm.get_cmap("coolwarm", 12)
    norm = cm.colors.Normalize(vmin=vmin, vmax=vmax)
    ### add colorbar
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    # empty array for the data range
    sm.A = []
    # prepare space for colorbar
    divider = make_axes_locatable(ax)
    size = "5%"
    cax = divider.append_axes("right", size = size, pad = .1)
    # add colorbar to figure
    cbar = fig.colorbar(sm, cax=cax, cmap= cmap)
    cbar.ax.tick_params(labelsize=18)
    vals = list(cbar.ax.get_yticks())
    vals.append(vmax)
    cbar.ax.set_yticklabels(vals)#[int(x) for x in vals])
    cbar.ax.set_ylabel("Normalized "+key + "\n(Feb 2020 = 1)", fontsize=20)
    
    df = n_u_data[n_u_data.index.get_level_values("date") == date]#.dropna(axis=0)
    df.plot(ax=ax, cax=ax, column = key,
            vmin=vmin, vmax=vmax,
            cmap=cmap, legend=False, 
            linewidth = .5, edgecolor="lightgrey",norm=norm)
    ax.set_title(title_date)
    plt.show()
    pp.savefig(fig, bbox_inches = "tight")
    plt.close()

 # close pdf   
pp.close()

We can see that unemployment rates tended to increase in regions hardest hit by COVID-19 early on. While the correlation is not perfect, the jump in unemployment in the northeast, including New Jersey, and in Michigan seem to follow this trend. Still, other factors matter. For example, the fall in oil prices seems to have impacted western Texas and western North Dakota. And although spread was limited on the West Coast, especially in California, a relatively strict lockdown there seems to be associated with higher levels of unemployment.

The interested student would benefit from running a panel regression [see Chapter 8](https://github.com/jlcatonjr/Learn-Python-for-Stats-and-Econ/blob/master/Textbook/Chapter%208%20-%20Advanced%20Data%20Analysis.ipynb) that includes factors like the stringency of lockdown and the severity of spread of COVID-19, along with other variables relevant to the level of unemployment, to estimate the effects of these elements on the overall level of unemployment.